# Creo un nuovo dataset 


---
## Analisi dei commenti di un ristorante

### Pierluigi - Lazio

URL di riferimento: https://www.tripadvisor.it/Restaurant_Review-g187791-d716359-Reviews-Pierluigi-Rome_Lazio.html

Home --> Ristoranti --> Firenze --> Best_ranking--> SENZ_ALTRO_Bistrot

In [1]:
# IMPORT NECESSARI
import requests
from bs4 import BeautifulSoup

print("Imported")

Imported


In [2]:
page_URL = "https://www.tripadvisor.it/Restaurant_Review-g187791-d716359-Reviews-Pierluigi-Rome_Lazio.html"

data = requests.get(page_URL)
clear_data = BeautifulSoup(data.content, "lxml")

print("Done")

Done


---
### STRUTTURA HTML DEL SITO 

#### REVIEW
`<p class="partial_entry"> qui c'è la review </p>`

#### N° TOTALE DI REVIEWS
`<a data-page-number="25" ...>
    25
 </a>`
 
#### NAMES
`<span class="expand_inline scrname" …>AlexandraCapeTown</span>`

#### RATINGS
`<span class="ui_bubble_rating bubble_30"></span>`

---
### PRIMO SCRAPING (sulla prima pagina)

Ottengo i nomi, i commenti, le review e da dove è stato scritto il commento per quanto rigurda la prima pagina. Ciò è necessario, in quanto, dalla seconda in poi l'URL cambia, permettendoci di iterare.

In [3]:
# OTTENGO UNA LISTA DI DIV CHE CONTENGONO NOME, RECENSIONE E VOTO

all_reviews = []
all_names = []
all_ratings = []
all_mobile = []

review_container = clear_data.findAll("div", attrs={"class":"review-container"})

for container in review_container:
  
    ### REVIEW ###
    review = container.find('p', attrs={'class': 'partial_entry'})
    if review:
        all_reviews.append(review.text)
        
    ### NAME ###
    name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
    if name:
      all_names.append(name.text)
    else:
      all_names.append('NO_NAME')
    
    ### RATING ###
    rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
    all_ratings.append(rating['class'][1][7:8])
    
    ### MOBILE ###
    mobile = container.find('span', attrs={'class': 'viaMobile'})
    if mobile:
      all_mobile.append('Mobile')
    else:
      all_mobile.append('PC')
    
    
################################################################################################################################
        
print("Executed")
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_ratings))

Executed
Nomi:  10
Commenti:  10
Voti:  10
Scritto da:  10


### SECONDO SCRAPING (dalla seconda pagina in poi)

Notiamo dall'URL che è possibile iterare sulle pagine, ottenendo i dati!

In [4]:
from tqdm import tqdm

URL_COMPLETO = 'https://www.tripadvisor.it/Restaurant_Review-g187791-d716359-Reviews-or10-Pierluigi-Rome_Lazio.html'

URL_p1 = 'https://www.tripadvisor.it/Restaurant_Review-g187791-d716359-Reviews-or'
URL_p2 = '0-Pierluigi-Rome_Lazio.html'
    
# OTTENGO I COMMENTI, NOMI E VOTI DALLA SECONDA PAGINA IN POI

total_pages = int(clear_data.select("a.pageNum.last.taLnk")[0].text) # 25 come intero

# per iterare dobbiamo partire dalla seconda pagina dei commenti, così nell'URL compaiono le cifre che ci servono
for r in tqdm(range(1,total_pages)):

    # OTTENGO L'URL DELLA PAGINA
    URL_temp = URL_p1 + str(r) + URL_p2

    # OTTENGO I DATI E CI APPLICO BEATIFULSOUP
    data = requests.get(URL_temp)
    clear_data = BeautifulSoup(data.content, "lxml")
    

    review_container = clear_data.findAll("div", attrs={"class":"review-container"})

    for container in review_container:
  
      ### REVIEW ###
      review = container.find('p', attrs={'class': 'partial_entry'})
      if review:
          all_reviews.append(review.text)

      ### NAME ###
      name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
      if name:
        all_names.append(name.text)
      else:
        all_names.append('NO_NAME')

      ### RATING ###
      rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
      all_ratings.append(rating['class'][1][7:8])

      ### MOBILE ###
      mobile = container.find('span', attrs={'class': 'viaMobile'})
      if mobile:
        all_mobile.append('Mobile')
      else:
        all_mobile.append('PC')
    
        
################################################################################################################################
        
print("Executed")
print('total_pages: ', total_pages)
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_mobile))

100%|██████████| 119/119 [02:43<00:00,  1.40s/it]

Executed
total_pages:  120
Nomi:  1195
Commenti:  1195
Voti:  1195
Scritto da:  1195


In [0]:
# display(all_reviews)

In [0]:
# print(all_names)

In [0]:
#print(all_ratings, '\n')
#print(len(all_ratings))

In [0]:
#display(all_mobile)

#### Problema recensioni troppo lunghe
Alla fine di alcune recensioni troviamo un '...Più', che non possiamo espandere. Data la mole di dati, decidiamo dunque di non considerarle.

In [9]:
all_reviews_filtered = []
counter_reviews_scartate = 0;
counter_reviews_da_usare = 0;

for review in all_reviews:
    if '...Più' in review:
        all_reviews_filtered.append('RECENSIONE_TROPPO_LUNGA')
        counter_reviews_scartate+=1;
    else:
        all_reviews_filtered.append(review)
        counter_reviews_da_usare+=1;
        
        
print(
    'Totale recensioni:', counter_reviews_scartate + counter_reviews_da_usare, '\n',
    'Recensioni analizzabili:', counter_reviews_da_usare, '\n',
    'Recensioni scartate:', counter_reviews_scartate, '\n')

display(all_reviews_filtered[:10])

Totale recensioni: 1195 
 Recensioni analizzabili: 814 
 Recensioni scartate: 381 



['RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'Qualità del pesce eccellente, piatti perfetti, personale gentile e disponibile, frequentato bene, prezzi importanti ma in linea con la qualità (essendo a Roma) merita sicuramente una visita.']

---
### Analisi dei nomi
Cerchiamo di capire se chi ha commentato è un uomo (M) o una donna (F).

#### Elimino spazi e numeri
Ho bisogno di tenere solo le informazioni riguardanti il nome, elimino il resto.

In [10]:
import re

all_names_filtered = []

regex = re.compile('[^a-zA-Z]')

for name in all_names:
    
    if ' ' in name:                           # se c'è uno spazio nel nome
        space = name.index(' ')               # ne trovo l'indice
        name = regex.sub('', name[:space])    # effettuo un controllo su tutto ciò che non è alfabeto fino a quell'indice
        all_names_filtered.append(name)       # e aggiungo la stringa
        
    else:
        name = regex.sub('', name)            # in caso contrario effettuo un controllo su tutto ciò che non è alfabeto
        all_names_filtered.append(name)       # e aggiungo la stringa
        
print(all_names, '\n')
print(all_names_filtered)

['NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'GuglielmoG85', 'KnightChevalier', 'Emanuele M', 'Consuelo N', 'MLuisaRav', 'nick_cassiusclay', 'maestroprimoconventi', 'ducagio', 'diegobon2013', 'michelebU4879RP', 'laviniap2014', 'Gian Luca Attol... A', 'Giovanna V', 'Bratislao', 'ottacarlotta', 'Alessandro P', 'MatteaV', 'Amarillide1', 'mgabriellam2017', 'Abrahamkatt', 'Cinziaver', 'giorgione1971', 'Ivanaxxx', 'roberto m', 'jamiarri', 'L0rdn40', 'laura m', 'brightly60', 'serdavi1', 'piedone51', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'Giuseppe P', 'Andrea G', 'dadahtml-687', 'Bestgustibus', 'maxigigliucci', 'francescomarzo68', 'EMMAT01', '854enricoz', '388Catherine388', 'Brunogrjjfjdnfjfj', 'Apicio82', 'maracaterina', 'Carlox92', 'gioRome', 'W5356PQclaudiab', 'massimo506', 'Ale_Sa_75', 'Filippo R', 'Asia16488', 'Andrea R', 'mafriroma', '254vierad', 'Boll

#### Utilizzo gender_guesser
Infine, posso utilizzare la libreria gender_guesser.

NB: Genderize ti permette di analizzare 1000 nomi al giorno.

In [11]:
!pip install gender_guesser

In [12]:
import gender_guesser.detector as gender
d = gender.Detector()

names_with_gender = []

for name in all_names_filtered:
    temp_gender = d.get_gender(name)
    names_with_gender.append(temp_gender)
  
print(names_with_gender)

['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'female', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'female', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 'unknown', 'unknown', 'female', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unkn

In [13]:
all_genders = []
i = 0

for guess in names_with_gender:
    if guess == 'male' or guess == 'mostly_male':
        all_genders.append('M')
    elif guess == 'female' or guess == 'mostly_female':
        all_genders.append('F')
    else:
        all_genders.append('UNKNOWN')

    
print(all_genders)

['UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'F', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'F', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 

---
## Salviamo ora i dati ottenuti in una tabella Excel
Così non dovremo ogni volta rieseguire tutto il codice



### Controlliamo che ci siano lo stesso numero di dati
Potremo così creare una tabella sfruttando Pandas

In [14]:
print('Nomi: ', len(all_names_filtered))
print('Commenti: ', len(all_reviews_filtered))
print('Voti: ', len(all_ratings))
print('Mobile: ', len(all_mobile))

Nomi:  1195
Commenti:  1195
Voti:  1195
Mobile:  1195


### Eliminiamo le recensioni troppo lunghe e quelle di cui non sappiamo il genere del commentatore
Creo una lista di liste [ ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ... ]

In [15]:
complete_list = []

i = len(all_names_filtered)

for indice in range(i):
  if all_genders[indice] != 'UNKNOWN' and all_reviews_filtered[indice] != 'RECENSIONE_TROPPO_LUNGA':
    temp = [all_names_filtered[indice], all_genders[indice], all_reviews_filtered[indice], all_ratings[indice], all_mobile[indice]]
    complete_list.append(temp)
    
print('DATI ANALIZZABILI: ', len(complete_list))
display(complete_list[:5])


DATI ANALIZZABILI:  172


[['Giuseppe',
  'M',
  "Consiglio di prenotare. Storico ristorante, assai gradevole anche all'aperto, nella piazza pedonale. Ottimo pesce, con porzioni regolari con servizio decisamente professionale. Piatti tutti ben elaborati e di alta qualità. Una chicca finale inperdibile: carpaccio di ananas.",
  '4',
  'Mobile'],
 ['Catherine',
  'F',
  'Ottimo ristorante, pesce freschissimo, lista dei vini notevole e staff preparatissimo e amichevole. Consiglio vivamente la catalana di gamberi...la migliore che abbia mai mangiato!!!!',
  '5',
  'Mobile'],
 ['Maurizio',
  'M',
  'Cenato in questo locale alla fine di tre intensi giorni di lavoro. Conosciuto il titolare e fatte quattro chiacchere molto ma molto interessanti ed utili. Lorenzo e’ persona intelligente e di una disponibilita’ unica. Location elegante, grande cantina e cucina ottima. Veramente molto positivo. Bravi.',
  '5',
  'Mobile'],
 ['Angela',
  'F',
  'Siamo ritornati di recente in questo ristorante e ancora una volta abbiamo sod

### Creiamo la tabella

In [0]:
# !pip install pandas
# !pip install xlsxwriter
import pandas as pd

In [0]:
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Nomi': [elemento[0] for elemento in complete_list], 
                   'Genere': [elemento[1] for elemento in complete_list], 
                   'Recensioni': [elemento[2] for elemento in complete_list],
                   'Voti': [elemento[3] for elemento in complete_list],
                   'Scritta da': [elemento[4] for elemento in complete_list]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('RISTORANTE_16.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [18]:
partial_dataframe = pd.read_excel("RISTORANTE_16.xlsx")

display(partial_dataframe)

,Genere,Nomi,Recensioni,Scritta da,Voti
0,M,Giuseppe,"Consiglio di prenotare. Storico ristorante, as...",Mobile,4
1,F,Catherine,"Ottimo ristorante, pesce freschissimo, lista d...",Mobile,5
2,M,Maurizio,Cenato in questo locale alla fine di tre inten...,Mobile,5
3,F,Angela,Siamo ritornati di recente in questo ristorant...,Mobile,5
4,F,Michelina,"Cibo assolutamente ordinario, servizio lento e...",PC,1
5,M,Federico,"buoni i crudi,ottimo il vino. Critico sulla pa...",PC,5
6,M,Domenico,"Ho detto tutto nel titolo, abbiamo mangiato be...",Mobile,4
7,M,Vito,Sono stato a cena con la mia compagna. Ambient...,PC,1
8,F,Margherita,Pierluigi è il ristorante!!! Tutti i crudi di ...,Mobile,5
9,F,Paola,Location stupenda nella cinquecentesca piazza ...,PC,4
